In [1]:
# Установка transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
from itertools import groupby

In [4]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
from datetime import datetime, timedelta

from catboost import CatBoostClassifier, Pool

In [5]:
#Считывание данных в DataFrame 

train= pd.read_csv('train_predobr_claster_vverh.csv', sep=';', index_col=None)
test= pd.read_csv('test_predobr_after7200_claster.csv', sep=';', index_col=None)
tabliza= pd.read_csv('tabliza_clasters_vverh.csv', sep=';', index_col=None)

In [6]:
train

,Номер кластера исходный,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,count_comments,id,assignee_id,creator_id,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Text_Summa,claster_centr_ishodn,mean_words_comment,Mesto_in_project
0,2,0.018732,14400,5,0,0,819947,93,93,"UI tests of the section ""Personal account""",2019,10,1,6,2,"UI tests of the section ""Personal account""",17134,0.0,0.038462
1,2,0.063944,7800,5,475,2,819933,207,1,Make a division by mobile operators,2019,10,1,6,2,Make a division by mobile operators Added the ...,17134,16.0,0.153846
2,2,0.003052,17580,5,17,2,819924,264,207,Doesn't work with cdr_calc_durations,2019,10,1,8,2,Doesn't work with cdr_calc_durations [~mursidi...,17134,30.0,0.230769
3,2,0.036756,22500,5,0,0,819921,94,94,Fix crash addons (NegativeArraySizeException),2019,10,1,11,2,Fix crash addons (NegativeArraySizeException),17134,0.0,0.250000
4,2,0.055723,9000,5,0,0,819910,1,1,Checking the client's balance when changing th...,2019,10,2,13,3,Checking the client's balance when changing th...,17134,0.0,0.326923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,2,0.035994,11880,48,0,0,688457,1,1,Investigate ANTMAN and find working CODs,2020,6,12,18,5,Investigate ANTMAN and find working CODs,17134,0.0,5.113941
4269,0,0.016286,67080,48,0,1,688422,1,1,Fix update script bug with JackfruitUsers,2020,6,12,20,5,Fix update script bug with JackfruitUsers Comm...,64703,62.0,5.194370
4270,2,0.018732,14400,48,0,1,688417,1,116,Teachers page returns 500 error,2020,6,15,9,1,Teachers page returns 500 error fixed. This is...,17134,11.0,5.207775
4271,2,0.061006,26040,48,0,0,688414,1,116,Make a test to check teachers pages for VN & TH,2020,6,15,9,1,Make a test to check teachers pages for VN & TH,17134,0.0,5.214477


In [7]:
test

,id,Id_razn,mean_words_comment,count_comments,project_id,Mesto_in_project,Year,Month,Day,Hour,DayOfWeek,assignee_id,creator_id,summary_to_eng,Text_Summa
0,675975,478,62.000000,8,5,8.884615,2020,1,15,9,3,112,112,Download all mp3 files of calls to disk,Download all mp3 files of calls to disk Resear...
1,675972,721,16.500000,4,5,8.903846,2020,1,15,9,3,112,112,Delete mp3 call files from the server before 0...,Delete mp3 call files from the server before 0...
2,675965,0,7.000000,1,5,8.961538,2020,1,16,4,4,94,54,Remove the tariff selection window and transfe...,Remove the tariff selection window and transfe...
3,675949,0,0.000000,0,5,9.153846,2020,1,17,8,5,54,54,Clear the project backlog of irrelevant tasks,Clear the project backlog of irrelevant tasks
4,675938,13,45.500000,2,5,9.269231,2020,1,20,4,1,94,54,Attach Firebase and GitLab bots to NambaTaxi S...,Attach Firebase and GitLab bots to NambaTaxi S...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,678981,20,34.833333,6,48,1.340483,2020,6,29,15,1,1,1,Logic of countdown clock in Payment Page,Logic of countdown clock in Payment Page [~sar...
485,688228,8,35.333333,3,48,5.623324,2020,6,30,15,2,82,82,Restoration of the monitoring and log system,Restoration of the monitoring and log system P...
486,688157,0,0.000000,0,48,5.757373,2020,7,8,0,3,82,82,Deploy EVN and ETL,Deploy EVN and ETL
487,688147,0,41.000000,1,48,5.770777,2020,7,9,3,4,82,82,Prepare documentation for Magento,Prepare documentation for Magento * Prepare in...


In [8]:
tabliza

,claster_centr,Номер кластера
0,64703,0
1,7014000,1
2,17134,2
3,3393960,3
4,1223520,4
5,196036,5
6,1955910,6
7,530386,7


In [9]:
train.columns

Index(['Номер кластера исходный', 'Расстояние до центра кластера',
       'overall_worklogs', 'project_id', 'Id_razn', 'count_comments', 'id',
       'assignee_id', 'creator_id', 'summary_to_eng', 'Year', 'Month', 'Day',
       'Hour', 'DayOfWeek', 'Text_Summa', 'claster_centr_ishodn',
       'mean_words_comment', 'Mesto_in_project'],
      dtype='object')

In [10]:
# выбираем поля для нейросети СatBoost по бинарной классификации

X_train=train[['project_id', 'assignee_id', 'creator_id', 'summary_to_eng',
        'Year', 'Month', 'Day', 'Hour', 'DayOfWeek', 'Mesto_in_project',
       'Id_razn', 'mean_words_comment', 'count_comments', 'Text_Summa' ]]
y_train=train[['Номер кластера исходный']]


testnet=test[['project_id', 'assignee_id', 'creator_id', 'summary_to_eng',
        'Year', 'Month', 'Day', 'Hour', 'DayOfWeek','Mesto_in_project',
       'Id_razn', 'mean_words_comment', 'count_comments','Text_Summa' ]]

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4273 entries, 0 to 4272
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   project_id          4273 non-null   int64  
 1   assignee_id         4273 non-null   int64  
 2   creator_id          4273 non-null   int64  
 3   summary_to_eng      4273 non-null   object 
 4   Year                4273 non-null   int64  
 5   Month               4273 non-null   int64  
 6   Day                 4273 non-null   int64  
 7   Hour                4273 non-null   int64  
 8   DayOfWeek           4273 non-null   int64  
 9   Mesto_in_project    4273 non-null   float64
 10  Id_razn             4273 non-null   int64  
 11  mean_words_comment  4273 non-null   float64
 12  count_comments      4273 non-null   int64  
 13  Text_Summa          4273 non-null   object 
dtypes: float64(2), int64(10), object(2)
memory usage: 467.5+ KB


In [12]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{0: 0.9836556169429097,
 1: 534.125,
 2: 0.1475075945871306,
 3: 534.125,
 4: 106.825,
 5: 6.594135802469136,
 6: 133.53125,
 7: 31.419117647058822}

In [13]:
#Обучаем 

pool_train_solution = Pool(X_train, y_train, cat_features = ['project_id', 'assignee_id', 'creator_id','Year', 'Month', 'Day', 'Hour', 'DayOfWeek'], text_features = ['summary_to_eng', 'Text_Summa'])
pool_test_solution = Pool(testnet,cat_features = ['project_id', 'assignee_id', 'creator_id','Year', 'Month', 'Day', 'Hour', 'DayOfWeek'], text_features = ['summary_to_eng', 'Text_Summa'])

model_test = CatBoostClassifier(iterations=1000,  task_type='GPU',depth=12, verbose =True, random_seed=24,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='MultiClassOneVsAll', class_weights=class_weights)
model_test.fit(pool_train_solution)

0:	learn: 0.7991376	total: 432ms	remaining: 7m 11s
1:	learn: 0.8443455	total: 749ms	remaining: 6m 13s
2:	learn: 0.8606589	total: 1.05s	remaining: 5m 48s
3:	learn: 0.8695450	total: 1.35s	remaining: 5m 36s
4:	learn: 0.8732859	total: 1.66s	remaining: 5m 29s
5:	learn: 0.8731352	total: 1.9s	remaining: 5m 14s
6:	learn: 0.8715913	total: 2.16s	remaining: 5m 7s
7:	learn: 0.8709973	total: 2.46s	remaining: 5m 4s
8:	learn: 0.8737720	total: 2.74s	remaining: 5m 2s
9:	learn: 0.8686481	total: 3.01s	remaining: 4m 58s
10:	learn: 0.8817222	total: 3.29s	remaining: 4m 55s
11:	learn: 0.8824015	total: 3.56s	remaining: 4m 53s
12:	learn: 0.8828387	total: 3.84s	remaining: 4m 51s
13:	learn: 0.8866952	total: 4.12s	remaining: 4m 50s
14:	learn: 0.8908289	total: 4.52s	remaining: 4m 56s
15:	learn: 0.8925436	total: 4.83s	remaining: 4m 57s
16:	learn: 0.8938901	total: 5.11s	remaining: 4m 55s
17:	learn: 0.8966763	total: 5.42s	remaining: 4m 56s
18:	learn: 0.8955943	total: 5.69s	remaining: 4m 53s
19:	learn: 0.8949143	total

In [14]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test.predict(X_train)
y_pred

array([[2],
       [2],
       [2],
       ...,
       [2],
       [2],
       [2]])

In [15]:
print("Значение метрики f1 на трейне по кластерам: ", f1_score(train['Номер кластера исходный'], y_pred, average='macro', zero_division = 0))

Значение метрики f1 на трейне по кластерам:  0.5871954444925575


In [16]:
train

,Номер кластера исходный,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,count_comments,id,assignee_id,creator_id,summary_to_eng,Year,Month,Day,Hour,DayOfWeek,Text_Summa,claster_centr_ishodn,mean_words_comment,Mesto_in_project
0,2,0.018732,14400,5,0,0,819947,93,93,"UI tests of the section ""Personal account""",2019,10,1,6,2,"UI tests of the section ""Personal account""",17134,0.0,0.038462
1,2,0.063944,7800,5,475,2,819933,207,1,Make a division by mobile operators,2019,10,1,6,2,Make a division by mobile operators Added the ...,17134,16.0,0.153846
2,2,0.003052,17580,5,17,2,819924,264,207,Doesn't work with cdr_calc_durations,2019,10,1,8,2,Doesn't work with cdr_calc_durations [~mursidi...,17134,30.0,0.230769
3,2,0.036756,22500,5,0,0,819921,94,94,Fix crash addons (NegativeArraySizeException),2019,10,1,11,2,Fix crash addons (NegativeArraySizeException),17134,0.0,0.250000
4,2,0.055723,9000,5,0,0,819910,1,1,Checking the client's balance when changing th...,2019,10,2,13,3,Checking the client's balance when changing th...,17134,0.0,0.326923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,2,0.035994,11880,48,0,0,688457,1,1,Investigate ANTMAN and find working CODs,2020,6,12,18,5,Investigate ANTMAN and find working CODs,17134,0.0,5.113941
4269,0,0.016286,67080,48,0,1,688422,1,1,Fix update script bug with JackfruitUsers,2020,6,12,20,5,Fix update script bug with JackfruitUsers Comm...,64703,62.0,5.194370
4270,2,0.018732,14400,48,0,1,688417,1,116,Teachers page returns 500 error,2020,6,15,9,1,Teachers page returns 500 error fixed. This is...,17134,11.0,5.207775
4271,2,0.061006,26040,48,0,0,688414,1,116,Make a test to check teachers pages for VN & TH,2020,6,15,9,1,Make a test to check teachers pages for VN & TH,17134,0.0,5.214477


In [17]:
# Проверим уровень схождения по r2 на трейне
train['Номер кластера']=y_pred
train=train.merge(tabliza, how="left", on="Номер кластера")
train['Номер кластера'].value_counts()

2    3904
0     300
5      47
7      15
6       3
4       3
3       1
Name: Номер кластера, dtype: int64

In [18]:
train['Номер кластера исходный'].value_counts()

2    3621
0     543
5      81
7      17
4       5
6       4
3       1
1       1
Name: Номер кластера исходный, dtype: int64

In [19]:
train

,Номер кластера исходный,Расстояние до центра кластера,overall_worklogs,project_id,Id_razn,count_comments,id,assignee_id,creator_id,summary_to_eng,...,Month,Day,Hour,DayOfWeek,Text_Summa,claster_centr_ishodn,mean_words_comment,Mesto_in_project,Номер кластера,claster_centr
0,2,0.018732,14400,5,0,0,819947,93,93,"UI tests of the section ""Personal account""",...,10,1,6,2,"UI tests of the section ""Personal account""",17134,0.0,0.038462,2,17134
1,2,0.063944,7800,5,475,2,819933,207,1,Make a division by mobile operators,...,10,1,6,2,Make a division by mobile operators Added the ...,17134,16.0,0.153846,2,17134
2,2,0.003052,17580,5,17,2,819924,264,207,Doesn't work with cdr_calc_durations,...,10,1,8,2,Doesn't work with cdr_calc_durations [~mursidi...,17134,30.0,0.230769,2,17134
3,2,0.036756,22500,5,0,0,819921,94,94,Fix crash addons (NegativeArraySizeException),...,10,1,11,2,Fix crash addons (NegativeArraySizeException),17134,0.0,0.250000,0,64703
4,2,0.055723,9000,5,0,0,819910,1,1,Checking the client's balance when changing th...,...,10,2,13,3,Checking the client's balance when changing th...,17134,0.0,0.326923,2,17134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268,2,0.035994,11880,48,0,0,688457,1,1,Investigate ANTMAN and find working CODs,...,6,12,18,5,Investigate ANTMAN and find working CODs,17134,0.0,5.113941,2,17134
4269,0,0.016286,67080,48,0,1,688422,1,1,Fix update script bug with JackfruitUsers,...,6,12,20,5,Fix update script bug with JackfruitUsers Comm...,64703,62.0,5.194370,2,17134
4270,2,0.018732,14400,48,0,1,688417,1,116,Teachers page returns 500 error,...,6,15,9,1,Teachers page returns 500 error fixed. This is...,17134,11.0,5.207775,2,17134
4271,2,0.061006,26040,48,0,0,688414,1,116,Make a test to check teachers pages for VN & TH,...,6,15,9,1,Make a test to check teachers pages for VN & TH,17134,0.0,5.214477,2,17134


In [20]:
print("Значение метрики r2 на трейне по секундам: ", r2_score(train['claster_centr'], train['overall_worklogs']))

Значение метрики r2 на трейне по секундам:  -0.33551222309078255


In [21]:
#Получение ответов
y_pred_solution = model_test.predict(pool_test_solution)

In [22]:
# Заводим результаты в тест
test['Номер кластера']=y_pred_solution
test['Номер кластера'].value_counts()

2    460
0     28
7      1
Name: Номер кластера, dtype: int64

In [23]:
# заводим центры кластеров в качестве ответа
test=test.merge(tabliza, how="left", on="Номер кластера")
test

,id,Id_razn,mean_words_comment,count_comments,project_id,Mesto_in_project,Year,Month,Day,Hour,DayOfWeek,assignee_id,creator_id,summary_to_eng,Text_Summa,Номер кластера,claster_centr
0,675975,478,62.000000,8,5,8.884615,2020,1,15,9,3,112,112,Download all mp3 files of calls to disk,Download all mp3 files of calls to disk Resear...,2,17134
1,675972,721,16.500000,4,5,8.903846,2020,1,15,9,3,112,112,Delete mp3 call files from the server before 0...,Delete mp3 call files from the server before 0...,2,17134
2,675965,0,7.000000,1,5,8.961538,2020,1,16,4,4,94,54,Remove the tariff selection window and transfe...,Remove the tariff selection window and transfe...,2,17134
3,675949,0,0.000000,0,5,9.153846,2020,1,17,8,5,54,54,Clear the project backlog of irrelevant tasks,Clear the project backlog of irrelevant tasks,2,17134
4,675938,13,45.500000,2,5,9.269231,2020,1,20,4,1,94,54,Attach Firebase and GitLab bots to NambaTaxi S...,Attach Firebase and GitLab bots to NambaTaxi S...,2,17134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,678981,20,34.833333,6,48,1.340483,2020,6,29,15,1,1,1,Logic of countdown clock in Payment Page,Logic of countdown clock in Payment Page [~sar...,2,17134
485,688228,8,35.333333,3,48,5.623324,2020,6,30,15,2,82,82,Restoration of the monitoring and log system,Restoration of the monitoring and log system P...,2,17134
486,688157,0,0.000000,0,48,5.757373,2020,7,8,0,3,82,82,Deploy EVN and ETL,Deploy EVN and ETL,2,17134
487,688147,0,41.000000,1,48,5.770777,2020,7,9,3,4,82,82,Prepare documentation for Magento,Prepare documentation for Magento * Prepare in...,2,17134


In [24]:
test_itog=test[['id','claster_centr']]
test_itog.columns=[['id','overall_worklogs']]
test_itog

,id,overall_worklogs
0,675975,17134
1,675972,17134
2,675965,17134
3,675949,17134
4,675938,17134
...,...,...
484,678981,17134
485,688228,17134
486,688157,17134
487,688147,17134


In [25]:
test_itog.to_csv('прогноз Пуассон аккураси сид 24.csv', sep=',', index=None)